In [1]:
%run ../__init__.py

## Data preprocessing with Tfidf

In [62]:
budget_train = pd.read_csv('../data/budget_df_cleaned2.csv', index_col=0)

In [3]:
# budget_df.head()

In [63]:
budget_test = pd.read_csv('../data/budget_test_clean.csv', index_col=0)

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (6,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [64]:
for col in budget_test[word_predictors[0:14]]:
    budget_test[col] = budget_test[word_predictors[0:14]].loc[:,[col]].fillna(' ')

In [65]:
for col in budget_df[word_predictors]:
    budget_train[col] = budget_train[word_predictors].loc[:,[col]].fillna(' ')

In [66]:
budget_test['FTE'] = np.clip(budget_test['FTE'],0,1)

In [81]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import Imputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import log_loss

In [55]:
label = LabelEncoder()

In [ ]:
class ItemSelector(BaseEstimator, TransformerMixin):
    """For data grouped by feature, select subset of data at a provided key.

    The data is expected to be stored in a 2D data structure, where the first
    index is over features and the second is over samples.  i.e.

    >> len(data[key]) == n_samples

    Please note that this is the opposite convention to scikit-learn feature
    matrixes (where the first index corresponds to sample).

    ItemSelector only requires that the collection implement getitem
    (data[key]).  Examples include: a dict of lists, 2D numpy array, Pandas
    DataFrame, numpy record array, etc.

    >> data = {'a': [1, 5, 2, 5, 2, 8],
               'b': [9, 4, 1, 4, 1, 3]}
    >> ds = ItemSelector(key='a')
    >> data['a'] == ds.transform(data)

    Parameters
     ----------
    key : hashable, required
        The key corresponding to the desired value in a mappable.
    """
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]
    

In [56]:
# 400,000+ examples, 14 features, return labels in {0, 1, 2}
X = budget_train[word_predictors] 
y = label.fit_transform(budget_train['Operating_Status'])

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = .2, random_state=42)
                                                

In [70]:
# feature union transformer list 
transformer_list = [
    ('Object_Description', Pipeline([
                ('selector', ItemSelector(key='Object_Description')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=100)),
            ])),
    ('Text_2', Pipeline([
                ('selector', ItemSelector(key='Text_2')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=100)),
            ])),
    ('SubFund_Description', Pipeline([
                ('selector', ItemSelector(key='SubFund_Description')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=100)),
            ])),
    ('Job_Title_Description', Pipeline([
                ('selector', ItemSelector(key='Job_Title_Description')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=100)),
            ])),
    ('Text_3', Pipeline([
                ('selector', ItemSelector(key='Text_3')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=100)),
            ])),
    ('Text_4', Pipeline([
                ('selector', ItemSelector(key='Text_4')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=100)),
            ])),
    ('Sub_Object_Description', Pipeline([
                ('selector', ItemSelector(key='Sub_Object_Description')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=100)),
            ])),
    ('Location_Description', Pipeline([
                ('selector', ItemSelector(key='Location_Description')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=100)),
            ])),
    ('Function_Description', Pipeline([
                ('selector', ItemSelector(key='Function_Description')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=100)),
            ])),
    ('Facility_or_Department', Pipeline([
                ('selector', ItemSelector(key='Facility_or_Department')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=100)),
            ])),
    ('Position_Extra', Pipeline([
                ('selector', ItemSelector(key='Position_Extra')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=100)),
            ])),
    ('Program_Description', Pipeline([
                ('selector', ItemSelector(key='Program_Description')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=100)),
            ])),
    ('Fund_Description', Pipeline([
                ('selector', ItemSelector(key='Fund_Description')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=100)),
            ])),
    ('Text_1', Pipeline([
                ('selector', ItemSelector(key='Text_1')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=100)),
            ])),
#     ('FTE', Pipeline([
# #                 ('selector', ItemSelector(key='FTE')),
# # #                 ('imputer', Imputer()),
# #             ])),
# #     ('Total', Pipeline([
# #                 ('selector', ItemSelector(key='Total')),
# # #                 ('imputer', Imputer()),
# #             ])),
   
]

# create pipeline
pipeline = Pipeline([
    ("union", FeatureUnion(transformer_list=transformer_list)),
    ("clf", LogisticRegression()),
])

In [71]:
X_test = budget_test

In [72]:
Ops_model = pipeline.fit(X_train, y_train)

In [73]:
Ops_model.score(X_train, y_train)

0.98528203959140714

In [74]:
predict_proba = Ops_model.predict_proba(X_val)

['Non-Operating', 'Operating, Not PreK-12', 'PreK-12 Operating']

In [82]:
log_loss(y_val, predict_proba)

0.047490405186274731

In [83]:
label.classes_

array(['Non-Operating', 'Operating, Not PreK-12', 'PreK-12 Operating'], dtype=object)